In [1]:
import tensorwrap as tf
import jax
from jax import jit
tf.config.set_visible_devices("gpu")

In [2]:
x = tf.range(1, 1e5)
x = tf.reshape(x, (1, -1))
y = x + 10
x, y = x/1e5, y/1e5

In [3]:
layer = tf.nn.layers.Dense(10)
layer.name[0]

'l'

In [4]:
layer.kernel_initializer

('glorot_uniform',)

In [5]:
model = tf.nn.Sequential([
    tf.nn.layers.Dense(10),
    tf.nn.layers.Dense(1)
])
model.compile(
    loss=tf.nn.losses.mae,
    optimizer=tf.nn.optimizers.gradient_descent(learning_rate = 1e-2),
)

In [6]:
layer = model.layers[0]

In [7]:
vars(model.trainable_layers['layer 0'])

{'dtype': None,
 'built': False,
 'name': 'layer 0',
 'trainable': True,
 'trainable_variables': {},
 'units': 10,
 'use_bias': (True,),
 'kernel_initializer': ('glorot_uniform',),
 'bias_initializer': ('zeros',),
 'kernel_regularizer': (None,),
 'bias_regularizer': (None,),
 'activity_regularizer': (None,),
 'kernel_constraint': (None,),
 'bias_constraint': (None,),
 'dynamic': False,
 'activation': <function tensorwrap.nn.activations.Activation.no_activate(inputs)>}

In [8]:
model.trainable_layers['layer 0'].trainable_variables

{}

In [9]:
%timeit model.fit(x, y, epochs=100, batch_size=32, verbose=0, hist_return=False)

120 ms ± 2.34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
